In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import time
from collections import Counter
import tensorflow as tf
from scipy import sparse

2023-01-12 19:00:52.755655: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-12 19:00:52.880476: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-12 19:00:52.880494: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-12 19:00:53.593902: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
data_folder = '/home/hanlinn/00.projects/tensorflow-prepare/example-file/bbc/'

train_csv_path = data_folder + 'BBC News Train.csv'
test_csv_path = data_folder + 'BBC News Test.csv'
sample_solutions_path = data_folder + 'BBC News Sample Solution.csv'



In [3]:
train_df = pd.read_csv(train_csv_path)
train_df.head()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business


In [4]:
test_df  = pd.read_csv(test_csv_path)
test_df.head()

,ArticleId,Text
0,1018,qpr keeper day heads for preston queens park r...
1,1319,software watching while you work software that...
2,1138,d arcy injury adds to ireland woe gordon d arc...
3,459,india s reliance family feud heats up the ongo...
4,1020,boro suffer morrison injury blow middlesbrough...


In [5]:
sample_solutions_pd = pd.read_csv(sample_solutions_path)
sample_solutions_pd.head()

,ArticleId,Category
0,1018,sport
1,1319,tech
2,1138,business
3,459,entertainment
4,1020,politics


In [6]:
train_df.dropna()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business
...,...,...,...
1485,857,double eviction from big brother model caprice...,entertainment
1486,325,dj double act revamp chart show dj duo jk and ...,entertainment
1487,1590,weak dollar hits reuters revenues at media gro...,business
1488,1587,apple ipod family expands market apple has exp...,tech


In [15]:
from sklearn.model_selection import train_test_split
train_sentence,val_sentence,train_label,val_labels = train_test_split(train_df["Text"].to_numpy(),
                                                                      train_df["Category"].to_numpy(),
                                                                      test_size=0.1, # dedicate 10% of samples to validation set
                                                                       random_state=42
                                                                        )

In [16]:
train_sentence.shape, val_sentence.shape

((1341,), (149,))

In [17]:
train_label.shape, val_labels.shape

((1341,), (149,))

In [18]:
train_label_cate = pd.get_dummies(train_label)

train_label_cate.head()

,business,entertainment,politics,sport,tech
0,0,0,1,0,0
1,0,0,0,1,0
2,1,0,0,0,0
3,0,0,0,0,1
4,0,0,0,1,0


In [19]:
trian_label_one_hot = train_label_cate.to_numpy()
trian_label_one_hot[3]

array([0, 0, 0, 0, 1], dtype=uint8)

In [20]:
val_label_cate = pd.get_dummies(val_labels)
val_label_cate.head()


,business,entertainment,politics,sport,tech
0,0,1,0,0,0
1,0,0,1,0,0
2,0,0,1,0,0
3,0,0,0,0,1
4,0,0,0,0,1


In [21]:
val_label_one_hot = val_label_cate.to_numpy()
val_label_one_hot[3]

array([0, 0, 0, 0, 1], dtype=uint8)

In [22]:

train_dataset = tf.data.Dataset.from_tensor_slices((train_sentence, trian_label_one_hot))
train_dataset =  train_dataset.shuffle(1341).batch(32).cache().prefetch(tf.data.AUTOTUNE)
valid_dataset = tf.data.Dataset.from_tensor_slices((val_sentence,val_label_one_hot))
valid_dataset = valid_dataset.batch(32).cache().prefetch(tf.data.AUTOTUNE)
print(f"Train : {train_dataset} \n"
      f"Test : {valid_dataset}")

Train : <PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None, 5), dtype=tf.uint8, name=None))> 
Test : <PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None, 5), dtype=tf.uint8, name=None))>


In [28]:

max_vocab = 10_000  # Maximum vocab size.
max_seq_len = 600  # Sequence length to pad the outputs to.

# Create the layer.
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=max_vocab,
    output_mode='int',
    output_sequence_length=max_seq_len)

# Now that the vocab layer has been created, call `adapt` on the
# text-only dataset to create the vocabulary. You don't have to batch,
# but for large datasets this means we're not keeping spare copies of
# the dataset.
vectorize_layer.adapt(train_sentence)

embedding_layers = tf.keras.layers.Embedding(input_dim=max_vocab,
                                     output_dim=5,
                                     embeddings_initializer="uniform",
                                     input_length = max_seq_len,
                                     name="embedding_layers")


In [29]:

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=5) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=3,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)

In [32]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = vectorize_layer(inputs)
x = embedding_layers(x)
x = layers.Conv1D(128, 5, padding = 'same', activation = 'elu')(x)
#x = layers.Conv1D(64, 5, padding = 'same', activation = 'elu')(x)
x = layers.GlobalMaxPool1D()(x)
#x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dense(5, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs, name="Conv")

model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00065),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])



Model: "Conv"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 600)              0         
 ectorization)                                                   
                                                                 
 embedding_layers (Embedding  (None, 600, 5)           50000     
 )                                                               
                                                                 
 conv1d_3 (Conv1D)           (None, 600, 128)          3328      
                                                                 
 global_max_pooling1d_3 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                              

In [34]:
from datetime import datetime

start = datetime.now()
history = model.fit(train_dataset,
                    epochs= 100,
                    validation_data=valid_dataset,
                    verbose=1)

end = datetime.now()

print(f"The time taken to train the model is :{end - start}")
results = model.evaluate(valid_dataset)


Epoch 1/100
42/42 [==============================] - 1s 31ms/step - loss: 1.1134 - accuracy: 0.4884 - val_loss: 1.1705 - val_accuracy: 0.4832
Epoch 2/100
42/42 [==============================] - 1s 33ms/step - loss: 0.9319 - accuracy: 0.6018 - val_loss: 1.0115 - val_accuracy: 0.5503
Epoch 3/100
42/42 [==============================] - 1s 34ms/step - loss: 0.7339 - accuracy: 0.6570 - val_loss: 0.7981 - val_accuracy: 0.7248
Epoch 4/100
42/42 [==============================] - 1s 31ms/step - loss: 0.5218 - accuracy: 0.8658 - val_loss: 0.5940 - val_accuracy: 0.8792
Epoch 5/100
42/42 [==============================] - 1s 27ms/step - loss: 0.3292 - accuracy: 0.9545 - val_loss: 0.4499 - val_accuracy: 0.9060
Epoch 6/100
42/42 [==============================] - 1s 29ms/step - loss: 0.1958 - accuracy: 0.9754 - val_loss: 0.3990 - val_accuracy: 0.8926
Epoch 7/100
42/42 [==============================] - 1s 27ms/step - loss: 0.1154 - accuracy: 0.9851 - val_loss: 0.3626 - val_accuracy: 0.8926
Epoch 

In [35]:

def calculate_accuracy_results(y_true, y_pred):
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    """
     Calculates model accuracy, precision, recall and f1 score of a binary classification model.

    Args:
        y_true: true labels in the form of a 1D array
        y_pred: predicted labels in the form of a 1D array

    Returns a dictionary of accuracy, precision, recall, f1-score.
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall and f1 score using "weighted average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted", zero_division= 1)
    model_results = {"accuracy": model_accuracy,
                      "precision": model_precision,
                      "recall": model_recall,
                      "f1": model_f1}
    return model_results

result_preds_probs = model.predict(val_sentence)
result_preds = tf.argmax(result_preds_probs, axis=1)
val_labels_encode = tf.argmax(val_label_one_hot ,axis=1)
results = calculate_accuracy_results( 
    y_true= val_labels_encode,
    y_pred = result_preds)

print(results)



5/5 [==============================] - 0s 7ms/step
{'accuracy': 90.60402684563759, 'precision': 0.9056693715568975, 'recall': 0.9060402684563759, 'f1': 0.9046773042677945}
